In [275]:
import requests
from selenium.webdriver import (Chrome, ChromeOptions)
import pymongo
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import yaml
import time

In [358]:
# Variables
path_to_fb_login_yaml = "/Users/jasonli/.secrets/facebook-web-scrape-cred.yaml"
my_profile_url = "https://www.facebook.com/jason.li.96930"

In [ ]:
mc = pymongo.MongoClient()
db = mc['my-facebook-webscrape']
fb_statuses = db['fb-statuses']

In [359]:
# Read FB Credentials
with open(path_to_fb_login_yaml, 'r') as stream:
    try:
        y = yaml.load(stream)
        my_password = y['password']
        my_email = y['email']
    except yaml.YAMLError as exc:
        print(exc)

In [368]:
# Login to FB in Selenium Browser
options = ChromeOptions();
options.add_argument("--disable-notifications");

browser = Chrome(options=options)
url = 'https://www.facebook.com/'
browser.get(url)

email = browser.find_element_by_id('email')
password = browser.find_element_by_id('pass')

email.send_keys(my_email)
password.send_keys(my_password)

browser.find_element_by_id("loginbutton").click()

In [369]:
# Navigate to the friends tab in your profile
browser.find_element_by_css_selector(f'a[href="{my_profile_url}"]').click()
time.sleep(1)

browser.find_element_by_css_selector('a[data-tab-key="friends"]').click()
time.sleep(1)

number_of_friends = int(browser.find_element_by_name('All Friends').find_elements_by_css_selector('span')[1].text)
print(number_of_friends)

416


In [296]:
# Create a dictionary of friends where key=friend_name and value=friend_profile_url 
SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

friends_profiles_dict = {}

while len(friends_profiles_dict.items()) < number_of_friends:

    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
    
    friend_items = browser.find_elements_by_css_selector('div[data-testid=friend_list_item]')
    
    if len(friends_profiles_dict.items()) > number_of_friends:
        break
    
    for friend_item in friend_items:
        profile_links = friend_item.find_elements_by_css_selector('a')
            
        for profile in profile_links:
            url = profile.get_attribute('href')
            
            if (
                my_profile_url not in url and 
                '?' in url and 
                'browse/mutual_friends/' not in url
            ):
                if not any(char.isdigit() for char in profile.text) or not profile.text:
                    if 'profile.php?id=' in url:
                        friends_profiles_dict[profile.text] = url
                    else:
                        s = url.split('?')[0]
                        friends_profiles_dict[profile.text] = s
            
    print('Creating friends dictionary... \n Friend count: ' +  str(len(friends_profiles_dict.items())) + ' friends.')

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print('Finished creating friends dictionary!')
        break
    last_height = new_height

21
41
81
101
120
139
158
175
194
214
234
254
254
293
313
332
352
372
391
402
402
402
402
402
402
402
402


In [373]:
friends_profiles_dict = {k: v for k, v in friends_profiles_dict.items() if k is not ''}
friends_profiles_dict

{'Adam Koehler': 'https://www.facebook.com/adam.koehler.50',
 'Adanimo Nrg': 'https://www.facebook.com/adam.arguello.35',
 'Adelina Manaut': 'https://www.facebook.com/adelinamanaut',
 'Alana McKnight': 'https://www.facebook.com/alanajoy93',
 'Alannah Rose': 'https://www.facebook.com/alannahnahnah',
 'Alex Beaton': 'https://www.facebook.com/alex.beaton.988',
 'Alex Chang': 'https://www.facebook.com/changAlang24',
 'Alex Grob': 'https://www.facebook.com/Grob.Alex',
 'Alex Keeno': 'https://www.facebook.com/akeeno11',
 'Alexander Cheung': 'https://www.facebook.com/alexander.cheung.564',
 'Alexander Hooton': 'https://www.facebook.com/alexander.hooton.5',
 'Alexander Pang': 'https://www.facebook.com/alexander.pang.5',
 'Alexander Robinson': 'https://www.facebook.com/PRIDE908',
 'Alexis Szigeti': 'https://www.facebook.com/alexismarieszigeti',
 'Alfred Quaicoe': 'https://www.facebook.com/fred.amissah',
 'Ali Patton': 'https://www.facebook.com/ali.patton.12',
 'Ally Holtz': 'https://www.faceboo

In [367]:
import random
import datetime

TypeError: Required argument 'month' (pos 2) not found

In [374]:
for name, url in friends_profiles_dict.items():
    # url = 'https://www.facebook.com/nick.dejesus.5'
    browser.get(url)

    # Create a dictionary of friends where key=friend_name and value=friend_profile_url 
    # SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    statuses_dict = {}

    while len(statuses_dict.items()) < 200:

        SCROLL_PAUSE_TIME = random.choice([1, 1.5, 2, 2.5, 3]) * (1 + random.random())

        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        posts = browser.find_elements_by_css_selector('div[id*=tl_unit]')
        for post in posts:
            try:
                post_time_element = post.find_element_by_css_selector('abbr')
                post_time = post_time_element.get_attribute('title')

                user_content_element = post.find_element_by_css_selector('div[class*=userContent]')
                para_elements = user_content_element.find_elements_by_css_selector('p')
                if len(para_elements) > 1:
                    text = ''
                    for para_element in para_elements:
                        text += para_element.text + ' '
                else:
                    text = para_elements.text
                statuses_dict[post_time] = text
            except:
                pass

        print('Creating statuses dictionary for ' + name + '... \n Status count: ' +  str(len(statuses_dict.items())) + ' statuses.')

        html = browser.page_source

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print('Finished creating statuses dictionary!')
            break
        last_height = new_height

    fb_statuses.insert_one(
        {
            'name': name,
            'url': url,
            'datetime': datetime.datetime.now(),
            'html': html,
            'statuses': statuses_dict
        }
    )

Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 1 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 2 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 2 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 2 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 3 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 3 statuses.
Creating statuses dictionary for Bradley Li... 
 Status count: 3 statuses.
Creating statuses diction

KeyboardInterrupt: 

In [371]:
statuses_dict

{'04/07/2018 10:17pm': 'I am now drinking Dark Souls while I play alcohol: www.twitch.tv/t7chicken Come watch me get rekt! ',
 '04/07/2018 9:00pm': "Sad Reacts only! 'I spent an hour crying over a goose just for this to happen' 😂😂 ",
 '04/08/2018 11:49pm': "Pariss Athena These kids are going nuts lol This'll be the last time Dad dresses as the Easter Bunny... 😳😂 ",
 '04/08/2018 7:42pm': 'I asked my bro to clip my toe nails for money. Me: 10 bucks Him: 20        ',
 '04/09/2018 9:09am': 'Ddaayyyuumm 😱😱😱 ***READY FOR SHOWINGS***\nFort Worth,TX 76126 | 9553 Bella Terra Dr.\nReady For Showings $7,000,000\n"PLEASE SHARE"           ',
 '04/10/2018 7:00am': "OMG this is so meta. I'm in tears lmao Positively Debordian. ",
 '04/10/2018 7:10pm': 'Nick DeJesus This one tho ',
 '04/11/2018 11:52pm': 'When I tell doctors I enjoy cleaning my ears with Q-Tips https://m.imgur.com/B5TBhZr ',
 '04/11/2018 4:32pm': '"how are you still alive?" Once in a blue moon 😂 ',
 '04/11/2018 9:44am': 'Nick DeJesus😂 

In [379]:
fb_statuses.insert_one(
        {
            'name': name,
            'url': url,
            'datetime': datetime.datetime.now(),
            'html': html,
            'statuses': statuses_dict
        }
    )
fb_statuses.find_one()

{'_id': ObjectId('5af0ec8f7a2f2ea00244feda'),
 'datetime': datetime.datetime(2018, 5, 7, 17, 17, 19, 623000),
 'html': '<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" id="facebook" class=""><head><meta charset="utf-8" /><meta name="referrer" content="origin-when-crossorigin" id="meta_referrer" /><link rel="preload" href="https://static.xx.fbcdn.net/rsrc.php/v3/yZ/r/guuj2-rjNLI.js" as="script" crossorigin="anonymous" /><link rel="preload" href="https://static.xx.fbcdn.net/rsrc.php/v3/yh/l/0,cross/iTiY4ncQFh4.css" as="style" crossorigin="anonymous" /><link rel="preload" href="https://static.xx.fbcdn.net/rsrc.php/v3/yX/l/0,cross/XqERxAyZZgA.css" as="style" crossorigin="anonymous" /><link rel="preload" href="https://static.xx.fbcdn.net/rsrc.php/v3/yR/l/0,cross/FyLOM-hJSNT.css" as="style" crossorigin="anonymous" /><link rel="preload" href="https://static.xx.fbcdn.net/rsrc.php/v3/yI/r/fFyKqmFK0NN.js" as="script" crossorigin="anonymous" /><link rel="preload" href="https:/